# Stimuler Exercise Recommendation System

### Specifics of the Problem Statement

1. **Feedback Categories**: As users converse with the app, feedback is given to each user utterance on 4 broad level categories - Grammar, Vocabulary, Pronunciation, and the content/fluency of speech. Scores and detailed feedback analysis are provided on each of these parts.

2. **Historical Data**: Historical data of each user’s past mistakes is used to suggest practice exercises. Assume the historical data of each user is available in a convenient format for building ML algorithms. Clarify data format assumptions in the final summary report.

3. **Exercise Recommendation Flow**: 
   - User starts a new conversation with the app.
   - At each user’s utterance, they are evaluated on the 4 categories (Grammar, Vocabulary, Pronunciation, Fluency/Content).
   - Dynamically decide whether to show the user a practice exercise or not (this algorithm already exists).
   - If a practice exercise is to be shown, generate the exercise based on the user’s past most common errors.

4. **Exercise Generation**: 
   - Decide whether the exercise should be for grammar, vocabulary, pronunciation, or content/fluency.
   - Generate the content of the practice exercise, which can be multimodal depending on the user’s demographics.

### Proposed Solution

#### Data Format Assumptions

- **User Data**: Contains demographic information such as country, age, English proficiency level, and interests.
- **Historical Data**: Contains past mistakes categorized into Grammar, Vocabulary, Pronunciation, and Fluency/Content, along with timestamps and context of errors.

#### Approach

1. **User Onboarding**: During onboarding, demographic information and user interests are collected. This information helps personalize the user experience by tailoring exercises to their background and preferences.

2. **Error Tracking**: User errors are continuously tracked and categorized into four main categories: Grammar, Vocabulary, Pronunciation, and Fluency/Content. This historical data is crucial for identifying patterns and common mistakes, which are used to generate relevant practice exercises.

3. **Dynamic Exercise Recommendation**:
   - **Identify Common Mistakes**: Historical error data is analyzed to identify the most frequent mistakes made by the user. This helps focus on areas where the user needs the most improvement.
   - **Decide Exercise Type**: Based on the identified common mistakes, the system decides whether the exercise should focus on grammar, vocabulary, pronunciation, or fluency.
   - **Personalize Exercise Content**: The content of the practice exercise is generated based on user demographics and interests. For example, culturally relevant content such as popular sitcoms or anime is used to make the exercises more engaging and relatable.

4. **Cold Start Scenario**: For new users, demographic information and initial interactions are used to make educated guesses about potential areas of improvement and interests. This can involve asking explicit questions during onboarding or using implicit feedback from initial conversations.

### Measures

To evaluate the effectiveness of the exercise recommendation system, the following measures can be used:

- **User Engagement**: Measure the time spent on exercises and the number of exercises completed. Higher engagement indicates that the exercises are relevant and interesting to the users.
- **Improvement in Scores**: Track the improvement in user scores over time in the four categories (Grammar, Vocabulary, Pronunciation, Fluency/Content). This helps in assessing whether the exercises are helping users improve their language skills.
- **User Satisfaction**: Collect feedback from users about the exercises. This can be done through surveys or in-app feedback mechanisms. High satisfaction scores indicate that users find the exercises helpful and enjoyable.

#### Metrics

To evaluate user performance, the following metrics are used:

- **Speech Rate**: Measures the number of words spoken per minute. A balanced speech rate indicates good fluency.
- **Pause Ratio**: Measures the ratio of pauses to speech. A lower pause ratio indicates better fluency and confidence.
- **Zero Crossing Rate**: Measures the rate at which the speech signal changes sign. A lower zero crossing rate indicates smoother speech and better pronunciation.

#### Scores

- **Vocabulary Score**: Assesses the richness of the user's vocabulary while also accounting for spelling accuracy. A higher score indicates a more extensive vocabulary with fewer spelling mistakes.
- **Grammar Score**: Assesses the correctness of the user's grammar. A higher score indicates fewer grammatical errors.
- **Pronunciation Score**: Evaluates the clarity and accuracy of the user's pronunciation. A higher score indicates better pronunciation.
- **Fluency Score**: Assesses the coherence and relevance of the user's speech. A higher score indicates better fluency and content quality.

#### Groq

Groq is used to leverage the capabilities of large language models (LLMs) for generating personalized and contextually relevant practice exercises. The LLM model used in this system is llama3-70b-8192, which provides advanced natural language understanding and generation capabilities. Groq's architecture allows for efficient interaction with the LLM, enabling the system to generate high-quality exercises in real-time.

#### LanguageTool API

The LanguageTool API is used for grammar and spelling checks. It provides detailed feedback on grammatical errors, helping users improve their writing and speaking skills. The API is integrated into the system to analyze user utterances and provide real-time feedback on grammar and vocabulary.

#### Summary

This solution outlines a dynamic exercise recommendation system for Stimuler, leveraging user demographic information and historical error data to personalize practice exercises. The approach ensures continuous adaptation to user interests and addresses the cold start scenario for new users effectively. By focusing on user engagement, improvement in scores, and user satisfaction, the system aims to provide a tailored and effective learning experience for users from diverse backgrounds.

#### Assumed Data Structure

In [1]:
# User Profile Data
user_profile = {
    'country': 'Japan',
    'age_band': '20s',  # e.g., 10s, 20s, 30s
    'proficiency_level': 'intermediate',  # e.g., beginner, intermediate, advanced
    'interests': ['anime', 'sitcoms'],  # e.g., anime, movies, sports, sitcoms
    'preferred_content_types': ['fill-in-the-blanks', 'memes']  # e.g., memes, fill-in-the-blanks, interactive quizzes, audio exercises
}

# User Performance Data
user_performance = {
    'grammar_score': 70,   # out of 100
    'vocabulary_score': 72,
    'pronunciation_score': 95,
    'fluency_score': 90,
}

In [2]:
import sounddevice as sd
import numpy as np
import time
import whisper
import scipy.io.wavfile as wav
import librosa

In [3]:
class AudioRecorder:
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
        
    def record_audio(self, duration):
        """Record audio for specified duration"""
        print(f"Recording for {duration} seconds...")

        recording = sd.rec(
            int(duration * self.sample_rate),
            samplerate=self.sample_rate,
            channels=1,
            dtype=np.float32
        )
        
        for i in range(duration):
            print(f"Recording: {i+1}/{duration} seconds")
            time.sleep(1)
            
        sd.wait()
        print("Recording finished!")
        return recording.flatten()
    
    def save_audio(self, audio_data, filename="recording.wav"):
        """Save audio to WAV file"""
        wav.write(filename, self.sample_rate, (audio_data * 32767).astype(np.int16))
        print(f"Audio saved to {filename}")

In [4]:
class SpeechAnalyzer:
    def __init__(self, model_size="base", sample_rate=16000):

        print("Loading Whisper model...")
        self.model = whisper.load_model(model_size)
        self.sample_rate = sample_rate
        print("Model loaded!")

    def analyze_speech(self, audio_data, duration):

        try:

            audio_data = whisper.pad_or_trim(audio_data)
            
            result = self.model.transcribe(audio_data)
            
            fluency = self._analyze_fluency(audio_data, result["text"], duration)
            pronunciation = self._analyze_pronunciation(audio_data)
            
            return {
                "transcription": result["text"],
                "language": result["language"],
                "fluency": fluency,
                "pronunciation": pronunciation
            }
        except Exception as e:
            print(f"Error during analysis: {e}")
            return None

    def _analyze_fluency(self, audio_data, text, duration):

        try:
            
            # Detect speech regions using energy-based splitting
            speech_intervals = librosa.effects.split(audio_data, top_db=30)
            print("Speech Intervals:")
            print(speech_intervals)
            total_speech_duration = sum((end - start) / self.sample_rate for start, end in speech_intervals)
            
            words = len(text.split())
            speech_rate = words / duration if duration > 0 else 0
            
            total_pause_duration = duration - total_speech_duration
            print(f"Total Pause Duration: {total_pause_duration}")
            print(f"Total Speech Duration: {total_speech_duration}")
            pause_ratio = total_pause_duration / duration if duration > 0 else 0

            onset_env = librosa.onset.onset_strength(y=audio_data, sr=self.sample_rate)
            onset_env = librosa.util.normalize(onset_env)
            rhythm_regularity = 1.0 - np.std(onset_env) / (np.mean(onset_env) + 1e-6)
            pace_consistency = 1.0 - np.std(np.diff(onset_env)) / (np.mean(onset_env) + 1e-6)

            # Fluency Score Calculation (out of 100)
            score = 100

            # Speech Rate Penalty (Normal rate: 2-2.5 words per second)
            if speech_rate < 2:  # Too slow
                score -= (2 - speech_rate) * 5
            elif speech_rate > 2.5:  # Too fast
                score -= (speech_rate - 2.5) * 5

            # Pause Ratio Penalty (Higher pause ratio means lower fluency)
            score -= pause_ratio * 40  

            score = max(0, min(100, score))
            
            return {
                "speech_rate": float(speech_rate),  # Words per second
                "pause_ratio": float(pause_ratio),  # Proportion of silence
                "rhythm_regularity": float(rhythm_regularity),  
                "pace_consistency": float(pace_consistency),  
                "fluency_score": float(score)  # Fluency score out of 100
            }
        except Exception as e:
            print(f"Error in fluency analysis: {e}")
            return None

    def _analyze_pronunciation(self, audio_data) -> dict:

        try:
            mfcc = librosa.feature.mfcc(y=audio_data, sr=self.sample_rate, n_mfcc=13)
            spectral_centroid = librosa.feature.spectral_centroid(y=audio_data, sr=self.sample_rate)
            zero_crossing_rate = librosa.feature.zero_crossing_rate(audio_data)
            
            mfcc_variance = np.var(mfcc)
            spectral_variance = np.var(spectral_centroid)
            zcr_mean = np.mean(zero_crossing_rate)
            
            optimal_zcr_min = 0.02
            optimal_zcr_max = 0.08
            
            # Pronunciation Score Calculation (out of 100)
            score = 100

            if zcr_mean < optimal_zcr_min:
                score -= (optimal_zcr_min - zcr_mean) * 500  # Penalize low ZCR
            elif zcr_mean > optimal_zcr_max:
                score -= (zcr_mean - optimal_zcr_max) * 500  # Penalize high ZCR
            
            score = max(0, min(100, score))
            
            return {
                "mfcc_variance": float(mfcc_variance),
                "spectral_variance": float(spectral_variance),
                "zero_crossing_rate": float(zcr_mean),
                "score": float(score)
            }
        except Exception as e:
            print(f"Error in pronunciation analysis: {e}")
            return None

In [ ]:
def analyze_voice_input(duration=5):
    """Record and analyze voice input"""
    # Initialize recorder and analyzer
    recorder = AudioRecorder()
    analyzer = SpeechAnalyzer()
    
    try:
        # Record audio
        print("\nStarting recording...")
        audio_data = recorder.record_audio(duration)
        
        # Save audio (optional)
        recorder.save_audio(audio_data, "last_recording.wav")
        
        # Analyze speech
        print("\nAnalyzing speech...")
        results = analyzer.analyze_speech(audio_data, duration)
        
        # Display results
        if results:
            print("\nResults:")
            print(f"Transcription: {results['transcription']}")
            print(f"Detected Language: {results['language']}")
            print("\nPronunciation:")
            for metric, value in results['pronunciation'].items():
                print(f"{metric}: {value:.2f}")
            print("\nFluency:")
            for metric, value in results['fluency'].items():
                print(f"{metric}: {value:.2f}")
            
        
    except Exception as e:
        print(f"Error: {e}")

# Test the function
analyze_voice_input(8) 

Loading Whisper model...
Model loaded!

Starting recording...
Recording for 8 seconds...
Recording: 1/8 seconds
Recording: 2/8 seconds
Recording: 3/8 seconds
Recording: 4/8 seconds
Recording: 5/8 seconds
Recording: 6/8 seconds
Recording: 7/8 seconds
Recording: 8/8 seconds
Recording finished!
Audio saved to last_recording.wav

Analyzing speech...
Speech Intervals:
[[20480 48128]
 [60416 69632]
 [70144 94208]]
Total Pause Duration: 4.192
Total Speech Duration: 3.808

Results:
Transcription:  Hello, my name is Saurri Dixit. I am a last year student at IIT Jamo.
Detected Language: en

Pronunciation:
mfcc_variance: 25791.00
spectral_variance: 1638675.98
zero_crossing_rate: 0.05
score: 100.00

Fluency:
speech_rate: 1.88
pause_ratio: 0.52
rhythm_regularity: -2.86
pace_consistency: -2.13
fluency_score: 78.41


In [14]:
def analyze_voice_input(duration=5):
    """Record and analyze voice input"""
    # Initialize recorder and analyzer
    recorder = AudioRecorder()
    analyzer = SpeechAnalyzer()
    
    try:
        # Record audio
        print("\nStarting recording...")
        audio_data = recorder.record_audio(duration)
        
        # Save audio (optional)
        recorder.save_audio(audio_data, "last_recording.wav")
        
        # Analyze speech
        print("\nAnalyzing speech...")
        results = analyzer.analyze_speech(audio_data, duration)
        
        # Display results
        if results:
            print("\nResults:")
            print(f"Transcription: {results['transcription']}")
            print(f"Detected Language: {results['language']}")
            print("\nPronunciation:")
            for metric, value in results['pronunciation'].items():
                print(f"{metric}: {value:.2f}")
            print("\nFluency:")
            for metric, value in results['fluency'].items():
                print(f"{metric}: {value:.2f}")
            
            # Calculate scores
            pronunciation_score = results['pronunciation']['score']
            fluency_score = results['fluency']['fluency_score']
            
            return results['transcription'], pronunciation_score, fluency_score
        
    except Exception as e:
        print(f"Error: {e}")
        return None, None

# Test the function
text, pronunciation_score, fluency_score = analyze_voice_input(8)
print(f"\nPronunciation Score: {pronunciation_score:.2f}")
print(f"Fluency Score: {fluency_score:.2f}")

Loading Whisper model...
Model loaded!

Starting recording...
Recording for 8 seconds...
Recording: 1/8 seconds
Recording: 2/8 seconds
Recording: 3/8 seconds
Recording: 4/8 seconds
Recording: 5/8 seconds
Recording: 6/8 seconds
Recording: 7/8 seconds
Recording: 8/8 seconds
Recording finished!
Audio saved to last_recording.wav

Analyzing speech...
Speech Intervals:
[[ 4608 29696]
 [38400 86016]]
Total Pause Duration: 3.4559999999999995
Total Speech Duration: 4.5440000000000005

Results:
Transcription:  Hello my name is Saurati Kshed. I am a last year student at IIT Jamun.
Detected Language: en

Pronunciation:
mfcc_variance: 31198.53
spectral_variance: 1729574.73
zero_crossing_rate: 0.05
score: 100.00

Fluency:
speech_rate: 1.88
pause_ratio: 0.43
rhythm_regularity: -2.30
pace_consistency: -1.56
fluency_score: 82.09

Pronunciation Score: 100.00
Fluency Score: 82.09


In [16]:
import requests
API_URL = "https://api.languagetool.org/v2/check"

In [17]:
def check_grammar_errors(text, lang="en-US"):

    params = {"text": text, "language": lang}
    response = requests.post(API_URL, data=params)

    if response.status_code == 200:
        result = response.json()
        matches = result.get("matches", [])

        grammar_errors = []
        for match in matches:
            rule_id = match.get("rule", {}).get("id", "")
            message = match.get("message", "")
            word = match.get("context", {}).get("text", "")[match.get("offset", 0): match.get("offset", 0) + match.get("length", 0)]
            
            # Exclude vocabulary errors like spelling and confused words
            if "MORFOLOGIK_RULE" not in rule_id and "CONFUSED_WORDS" not in rule_id:
                grammar_errors.append({
                    "word": word,
                    "suggestions": [sug["value"] for sug in match.get("replacements", [])],
                    "message": message
                })

        return grammar_errors
    else:
        return {"error": f"API request failed with status code {response.status_code}"}

sentences = [
    "She go to school every day.",  
    "He have been working since morning.",  
    "The cat is on table.",  
    "I has a pen.",  
    "He had a week grasp of the subject and made severel vocab mistakez."  
]

for sentence in sentences:
    print(f"\nChecking: {sentence}")
    errors = check_grammar_errors(sentence)

    if errors:
        print("Grammar Errors Found:")
        for err in errors:
            print(f" - Word: {err['word']}")
            print(f"   Suggestion(s): {', '.join(err['suggestions'])}")
            print(f"   Message: {err['message']}\n")
    else:
        print("No grammar errors found.")



Checking: She go to school every day.
Grammar Errors Found:
 - Word: go
   Suggestion(s): goes, went
   Message: The pronoun ‘She’ is usually used with a third-person or a past tense verb.


Checking: He have been working since morning.
Grammar Errors Found:
 - Word: have
   Suggestion(s): has, had
   Message: The pronoun ‘He’ is usually used with a third-person or a past tense verb.


Checking: The cat is on table.
No grammar errors found.

Checking: I has a pen.
Grammar Errors Found:
 - Word: has
   Suggestion(s): have
   Message: Possible agreement error — use the base form here.


Checking: He had a week grasp of the subject and made severel vocab mistakez.
Grammar Errors Found:
 - Word: week
   Suggestion(s): weak
   Message: Please check whether ‘weak’ (opposite of strong) might be the correct word here instead of ‘week’ (a period of seven days).



In [18]:
def check_vocabulary_errors(text, lang="en-US"):

    params = {"text": text, "language": lang}
    response = requests.post(API_URL, data=params)

    if response.status_code == 200:
        result = response.json()
        matches = result.get("matches", [])

        vocab_errors = []
        for match in matches:
            rule_id = match.get("rule", {}).get("id", "")
            message = match.get("message", "")
            word = match.get("context", {}).get("text", "")[match.get("offset", 0): match.get("offset", 0) + match.get("length", 0)]

            # Only include vocabulary-related errors (spelling & misused words)
            if "MORFOLOGIK_RULE" in rule_id or "CONFUSED_WORDS" in rule_id:
                vocab_errors.append({
                    "word": word,
                    "suggestions": [sug["value"] for sug in match.get("replacements", [])],
                    "message": message
                })

        return vocab_errors
    else:
        return {"error": f"API request failed with status code {response.status_code}"}
    
sentences = [
    "He had a week grasp of the subject and made severel vocab mistakez.",  
    "Their going to the store later.",  
    "The effect of the medcine is immediate.",  
    "He wrote a greate essay.",  
    "This is a correct sentence."  
]

for sentence in sentences:
    print(f"\nChecking: {sentence}")
    errors = check_vocabulary_errors(sentence)

    if errors:
        print("Vocabulary Errors Found:")
        for err in errors:
            print(f" - Word: {err['word']}")
            print(f"   Suggestion(s): {', '.join(err['suggestions'])}")
            print(f"   Message: {err['message']}\n")
    else:
        print("No vocabulary errors found.")



Checking: He had a week grasp of the subject and made severel vocab mistakez.
Vocabulary Errors Found:
 - Word: everel 
   Suggestion(s): several, severe, severely, severed, severer
   Message: Possible spelling mistake found.

 - Word: 
   Suggestion(s): mistake, mistaken, mistakes, mistaker
   Message: Possible spelling mistake found.


Checking: Their going to the store later.
No vocabulary errors found.

Checking: The effect of the medcine is immediate.
Vocabulary Errors Found:
 - Word: medcine
   Suggestion(s): medicine, med cine
   Message: Possible spelling mistake found.


Checking: He wrote a greate essay.
Vocabulary Errors Found:
 - Word: greate
   Suggestion(s): great, create, greater, grease, greats, grate, greave, Greater
   Message: Possible spelling mistake found.


Checking: This is a correct sentence.
No vocabulary errors found.


In [19]:
def calculate_vocabulary_score(text, lang="en-US"):

    params = {"text": text, "language": lang}
    response = requests.post(API_URL, data=params)

    if response.status_code == 200:
        result = response.json()
        matches = result.get("matches", [])

        vocab_errors = []
        error_length = 0
        total_length = len(text)

        for match in matches:
            rule_id = match.get("rule", {}).get("id", "")
            message = match.get("message", "")
            word = match.get("context", {}).get("text", "")[match.get("offset", 0): match.get("offset", 0) + match.get("length", 0)]

            # Only include vocabulary-related errors (spelling & misused words)
            if "MORFOLOGIK_RULE" in rule_id or "CONFUSED_WORDS" in rule_id:
                vocab_errors.append({
                    "word": word,
                    "suggestions": [sug["value"] for sug in match.get("replacements", [])],
                    "message": message
                })
                error_length += len(word)

        score = 100 - (error_length / total_length) * 100

        return score
    else:
        return {"error": f"API request failed with status code {response.status_code}"}
    



In [20]:
calculate_vocabulary_score("He had a week grasp of the subject and made severel vocab mistakez.")

89.55223880597015

In [21]:
def calculate_grammar_score(text, lang="en-US"):

    params = {"text": text, "language": lang}
    response = requests.post(API_URL, data=params)

    if response.status_code == 200:
        result = response.json()
        matches = result.get("matches", [])

        grammar_errors = []
        error_length = 0
        total_length = len(text)

        for match in matches:
            rule_id = match.get("rule", {}).get("id", "")
            message = match.get("message", "")
            word = match.get("context", {}).get("text", "")[match.get("offset", 0): match.get("offset", 0) + match.get("length", 0)]
            
            # Exclude vocabulary errors like spelling and confused words
            if "MORFOLOGIK_RULE" not in rule_id and "CONFUSED_WORDS" not in rule_id:
                grammar_errors.append({
                    "word": word,
                    "suggestions": [sug["value"] for sug in match.get("replacements", [])],
                    "message": message
                })
                error_length += len(word)

        score = 100 - (error_length / total_length) * 100

        return score
    else:
        return {"error": f"API request failed with status code {response.status_code}"}


In [22]:
calculate_grammar_score("He had a week grasp of the subject and made severel vocab mistakez.")

94.02985074626866

In [23]:
from IPython.display import display, clear_output
import ipywidgets as widgets

In [24]:
def App():
    def on_input_type_change(change):
        clear_output()
        if change['new'] == 'Voice Input':
            get_voice_input()
        else:
            get_text_input()
    
    def get_voice_input():
        voice_input, pronunciation_score, fluency_score = analyze_voice_input(8)

        # Update user_performance dictionary
        if 'pronunciation_score' in user_performance:
            user_performance['pronunciation_score'] = (user_performance['pronunciation_score'] + pronunciation_score) / 2
        else:
            user_performance['pronunciation_score'] = pronunciation_score
            
        if 'fluency_score' in user_performance:
            user_performance['fluency_score'] = (user_performance['fluency_score'] + fluency_score) / 2
        else:
            user_performance['fluency_score'] = fluency_score

        print("Your input:")
        print(voice_input)
        print("\nScores:")
        print(f"Pronunciation Score: {pronunciation_score:.2f}")
        print(f"Fluency Score: {fluency_score:.2f}")
        print("\nUpdated Performance:")
        print(f"Average Pronunciation Score: {user_performance['pronunciation_score']:.2f}")
        print(f"Average Fluency Score: {user_performance['fluency_score']:.2f}")
    
    def calculate_and_update_scores(text):
        # Calculate new scores
        vocab_score = calculate_vocabulary_score(text)
        grammar_score = calculate_grammar_score(text)
        
        # Update user_performance dictionary
        if 'vocabulary_score' in user_performance:
            user_performance['vocabulary_score'] = (user_performance['vocabulary_score'] + vocab_score) / 2
        else:
            user_performance['vocabulary_score'] = vocab_score
            
        if 'fluency_score' in user_performance:
            user_performance['grammar_score'] = (user_performance['grammar_score'] + grammar_score) / 2
        else:
            user_performance['grammar_score'] = grammar_score
            
        return vocab_score, grammar_score
    
    def get_text_input():
        text_input = widgets.Text(
            placeholder='Type your input here...',
            description='Input:',
            style={'description_width': 'initial'}
        )
        
        submit_button = widgets.Button(description='Submit')
        
        def on_submit(b):
            vocab_score, grammar_score = calculate_and_update_scores(text_input.value)
            display_output(text_input.value, vocab_score, grammar_score)
        
        submit_button.on_click(on_submit)
        
        display(widgets.VBox([text_input, submit_button]))
    
    def display_output(text, vocab_score, grammar_score):
        clear_output()
        print("Your input:")
        print(text)
        print("\nScores:")
        print(f"Vocabulary Score: {vocab_score:.2f}")
        print(f"Grammar Score: {grammar_score:.2f}")
        print("\nUpdated Performance:")
        print(f"Average Vocabulary Score: {user_performance['vocabulary_score']:.2f}")
        print(f"Average Grammar Score: {user_performance['grammar_score']:.2f}")
        
        retry_button = widgets.Button(description='Try Again')
        
        def on_retry(b):
            clear_output()
            start_app()
        
        retry_button.on_click(on_retry)
        display(retry_button)
    
    def start_app():
        input_type = widgets.RadioButtons(
            options=['Text Input', 'Voice Input'],
            description='Choose input type:',
            style={'description_width': 'initial'},
            value='Text Input'  # Set default value
        )
        
        input_type.observe(on_input_type_change, names='value')
        display(input_type)

        get_text_input()
    
    start_app()

App()

Your input:
He had a week grasp of the subject and made severel vocab mistakez.

Scores:
Vocabulary Score: 89.55
Grammar Score: 94.03

Updated Performance:
Average Vocabulary Score: 80.78
Average Grammar Score: 82.01


Button(description='Try Again', style=ButtonStyle())

In [28]:
App()

Loading Whisper model...
Model loaded!

Starting recording...
Recording for 8 seconds...
Recording: 1/8 seconds
Recording: 2/8 seconds
Recording: 3/8 seconds
Recording: 4/8 seconds
Recording: 5/8 seconds
Recording: 6/8 seconds
Recording: 7/8 seconds
Recording: 8/8 seconds
Recording finished!
Audio saved to last_recording.wav

Analyzing speech...
Speech Intervals:
[[11264 36864]
 [46592 70144]
 [70656 86528]]
Total Pause Duration: 3.936
Total Speech Duration: 4.064

Results:
Transcription:  Hello my name is Sohra Dixip, I am a last-year student at IIT Zomo.
Detected Language: en

Pronunciation:
mfcc_variance: 29939.19
spectral_variance: 1722176.22
zero_crossing_rate: 0.05
score: 100.00

Fluency:
speech_rate: 1.75
pause_ratio: 0.49
rhythm_regularity: -2.37
pace_consistency: -1.59
fluency_score: 79.07
Your input:
 Hello my name is Sohra Dixip, I am a last-year student at IIT Zomo.

Scores:
Pronunciation Score: 100.00
Fluency Score: 79.07

Updated Performance:
Average Pronunciation Score: 

In [29]:
import groq

#### Generating Exercise

In [35]:
def get_lowest_skill(user_performance):
    return min(user_performance, key=user_performance.get)

def generate_prompt(user_profile, user_performance):
    lowest_skill = get_lowest_skill(user_performance)
    interests = ', '.join(user_profile['interests'])
    preferred_formats = ', '.join(user_profile['preferred_content_types'])
    proficiency_level = user_profile['proficiency_level']

    
    prompt = (
        f"Generate an English language exercise for a user from {user_profile['country']} "
        f"in their {user_profile['age_band']} with an {proficiency_level} proficiency level. "
        f"The user prefers formats like {preferred_formats}. "
        f"The exercise should focus on improving {lowest_skill}, as it is their weakest area. "
        f"Make the exercise engaging by including references to {interests}. "
        f"Ensure the content is suitable for an {proficiency_level} learner."
    )
    return prompt

def generate_exercise(prompt):
    client = groq.Client(api_key="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")  
    response = client.chat.completions.create(
        model="llama3-70b-8192",  
        messages=[{"role": "system", "content": "You are an expert English teacher."},
                  {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

if __name__ == "__main__":
    
    prompt = generate_prompt(user_profile, user_performance)
    exercise = generate_exercise(prompt)
    
    print("Generated Exercise:\n")
    print(exercise)


Generated Exercise:

Here's a fun exercise to help improve vocabulary skills, tailored to a Japanese learner in their 20s with an intermediate proficiency level:

**Anime-themed Vocabulary Building Exercise**

**Fill in the Blanks: "Anime Characters' Catchphrases"**

Complete the sentences with the correct vocabulary words. Choose from the options provided.

1. In the anime "Naruto," Naruto Uzumaki's catchphrase is "Believe it!" which means to have _______________________ in someone or something.

A) confidence
B) skepticism
C) nostalgia
D) empathy

Answer: A) confidence

2. Light Yagami from "Death Note" is known for his _______________________ nature, always staying one step ahead of L.

A) meticulous
B) spontaneous
C) gullible
D) naive

Answer: A) meticulous

3. In "One Punch Man," Saitama's _______________________ attitude towards his superhero life is both relatable and hilarious.

A) nonchalant
B) enthusiastic
C) sarcastic
D) melancholic

Answer: A) nonchalant

**Meme Time!**

Id